In [1]:
#setup libraries
import numpy as np
import pandas as pd
import os, re, glob, sys
from skimage import io, filters, util, segmentation, morphology, measure, restoration, exposure
from skimage.registration import phase_cross_correlation
import matplotlib.pyplot as plt
#import imageio
from cellpose import models, plot
from scipy import stats, spatial, ndimage
from PIL import Image, ImageSequence
import tifffile

In [2]:
pipeline_name = "CtnK" #cellpose and KIT tracking
ch1_name = 'NR'
ch2_name = 'CC'
data_path = '/home/exacloud/gscratch/HeiserLab/images/'
#data_path = '/Users/dane/Documents/CellTrackingProjects/AU565/images/'
#data_path = '/Users/dane/Documents/CellTrackingProjects/HCC1143/images/'
plateID = 'HC01401'
#plateID = sys.argv[1]
well_index = 1
#well_index = int(sys.argv[2])

output_path = os.path.join(data_path+plateID,"Analysis",pipeline_name,"intermediate_files/")
registered_stacks_path = os.path.join(data_path+plateID,"Analysis","registered_stacks")
transformation_path = os.path.join(output_path,"transformations")
tracking_path = os.path.join(output_path,'tracking/')

#Only use subdirectories within the selected well
well_directory = sorted(glob.glob(os.path.join(data_path+plateID,"[A-Z][1-9]")))[well_index-1]
well = re.findall("[A-Z][1-9]$",well_directory)[0]
subdirectories = sorted(glob.glob(os.path.join(well_directory,"field_[1-9]")))

#### Registration method
If there is a registered red channel stack skip this step, otherwise:  

Image registration is performed on the red channel nuclear marker image stack using the python skimage phase_cross_correlation function to correct translations. The calculated shifts are then applied to all image stacks and saved for downstream usage. The image stacks are then cropped to their common areas so that no blank areas are analyzed.

In [ ]:

for subdir in subdirectories:
    field = re.findall("field_[1-9]",subdir)[0]
    field_num = re.findall("[0-9]", field)[0]
    reg_filename = os.path.join(registered_stacks_path,plateID+"_R_"+well+"_"+field_num+"_reg_stack.tif")
    # Only process the field-level image files if there is no registered stack
    if not os.path.exists(reg_filename):
        print("Gathering image stacks for "+subdir)
        #load and prepare red, green and phase channels. Scale for 8 bits but these are uint16 data types
        r_data_paths = glob.glob(os.path.join(subdir,"*_R_*m.tif"))
        r_time_slices = set()
        for data_paths in r_data_paths:
            r_time_slices.add(re.findall("..d..h..m", data_paths)[0])
        g_data_paths = glob.glob(os.path.join(subdir,"*_G_*m.tif"))
        g_time_slices = set()
        for data_paths in g_data_paths:
            g_time_slices.add(re.findall("..d..h..m", data_paths)[0])
        p_data_paths = glob.glob(os.path.join(subdir,"*_P_*m.tif"))
        p_time_slices = set()
        for data_paths in p_data_paths:
            p_time_slices.add(re.findall("..d..h..m", data_paths)[0])
        complete_time_slices = r_time_slices & g_time_slices & p_time_slices
        r_data_paths_c = []
        g_data_paths_c = []
        p_data_paths_c = []
        for time_slice in complete_time_slices:
            r_data_paths_c.append(os.path.join(data_path+plateID,well,field,plateID+"_R_"+well+"_"+field_num+"_"+time_slice+".tif"))
            g_data_paths_c.append(os.path.join(data_path+plateID,well,field,plateID+"_G_"+well+"_"+field_num+"_"+time_slice+".tif"))
            p_data_paths_c.append(os.path.join(data_path+plateID,well,field,plateID+"_P_"+well+"_"+field_num+"_"+time_slice+".tif"))
        img_r_ic = io.imread_collection(r_data_paths_c) # 3 dimensions : frames x width x height
        img_rs = np.stack(img_r_ic)

        img_g_ic = io.imread_collection(g_data_paths_c) # 3 dimensions : frames x width x height
        img_gs = np.stack(img_g_ic)

        img_p_ic = io.imread_collection(p_data_paths_c) # 3 dimensions : frames x width x height
        img_ps = np.stack(img_p_ic)
        
        #register using skimage skimage.registration phase_cross_correlation
        
        shifts = np.zeros([img_ps.shape[0], 2])
        img_rs_reg = img_rs
        img_gs_reg = img_gs
        img_ps_reg = img_ps
        print("calculating shifts")
        for i, image in enumerate(img_ps[range(img_ps.shape[0]-1)]): #loop through the first to the second to last image
            shift, error, diffphase = phase_cross_correlation(image, img_ps[i+1], normalization=None, upsample_factor=4)
            #print(str(i)+" shift: "+str(shift))
            shifts[i+1] = shifts[i]+shift #make shifts absolute, based on first image
            #print(str(i)+" absolute shift: "+str(shifts[i+1]))
        print("shifting image stacks")
        for i in range(img_ps.shape[0]):
            img_rs_reg[i] = ndimage.shift(img_rs[i], shifts[i], order=3, mode='constant', cval=0, prefilter=True)
            img_gs_reg[i] = ndimage.shift(img_gs[i], shifts[i], order=3, mode='constant', cval=0, prefilter=True)
            img_ps_reg[i] = ndimage.shift(img_ps[i], shifts[i], order=3, mode='constant', cval=0, prefilter=True)
            
        if not os.path.exists(transformation_path):
            os.makedirs(transformation_path)
        np.save(os.path.join(transformation_path,plateID+"_"+well+"_"+field+"_shifts.npy"), shifts)

        if not os.path.exists(output_path):
            os.makedirs(output_path)

        #assume transformations are in pixels and crop images to exclude areas that are outside of any registered image
        x_axis_length = img_rs_reg.shape[2]
        y_axis_length = img_rs_reg.shape[1]
        
        #crop each stack to the area that is common in all images after registration
        #use the min and max values in the transformation stack to define the common active area 
        x_max = shifts[:,1].max().astype("int")
        x_min = shifts[:,1].min().astype("int")
        y_max = shifts[:,0].max().astype("int")
        y_min = shifts[:,0].min().astype("int")
        
        #assume transformations are in pixels and crop images to exclude areas that are outside of any registered image
        x_axis_length = img_rs_reg.shape[2]
        y_axis_length = img_rs_reg.shape[1]
        print("Cropping to active areas for "+subdir)
        
        img_rs_reg_crop = img_rs_reg[:,np.max([0, y_max]):(y_axis_length+np.min([0,y_min])),np.max([0, x_max]):(x_axis_length+np.min([0,x_min]))]
        img_gs_reg_crop = img_gs_reg[:,np.max([0, y_max]):(y_axis_length+np.min([0,y_min])),np.max([0, x_max]):(x_axis_length+np.min([0,x_min]))]
        img_ps_reg_crop = img_ps_reg[:,np.max([0, y_max]):(y_axis_length+np.min([0,y_min])),np.max([0, x_max]):(x_axis_length+np.min([0,x_min]))]
        
        #combine phase and nuclear channels
        print("Saving stacks to disk for "+subdir)
        io.imsave(reg_filename, img_rs_reg_crop.astype(np.int16), plugin='tifffile', check_contrast=False)
        io.imsave(reg_filename.replace("_R_","_G_"), img_gs_reg_crop.astype(np.int16), plugin='tifffile', check_contrast=False)
        io.imsave(reg_filename.replace("_R_","_P_"), img_ps_reg_crop.astype(np.int16), plugin='tifffile', check_contrast=False)

#### Segment the nuclear fluorescent images using cellpose  
If mask files already exist, skip this step, otherwise:    
Load the registered red stack nuclear marker images  
Segment each image and only keep masks greater than a selected area  
Save the mask files as an image sequence  
For compatibility with the tracking method, save the masks and the nuclear intensity images as indivdual files  

#### Segmentation method
Images from the nuclear marker channel are segmented with the Cellpose cyto2 model. Cellpose parameters are diameter = 17, flow_threshold = .4 and cellprob_threshold = 0. After segmentations, objects with area less than 150 or greater than 500 are considered debris and are removed.

In [ ]:
def select_cellpose_n_diameter(x):
    return {
        'AU00601': 13,
        'AU01501': 13,
        'AU01601': 13,
        'AU01701': 13,
        'AU01702': 13,
        'AU01801': 13,
        'AU01802': 13,
        'AU01901': 13,
        'AU01902': 13,
        'AU02001': 13,
        'AU02002': 13,
        'AU02101': 13,
        'AU02201': 13,
        'AU02301': 13,
        'AU02201': 13,
        'AU02401': 13,
        'AU02501': 13,
        'AU02601': 13,
        'AU02701': 13,
        'AU02801': 13,
        'AU02901': 13,
        'AU03001': 13,
        'AU03101': 13,
        'AU03201': 13,
        'AU03301': 13,
        'AU03401': 13,
        'AU03501': 13,
        'AU03601': 13,
        'AU03701': 13,
        'HC00701': 17,'HC00801': 17,'HC00901': 17,'HC01001': 17, 'HC01301': 17,'HC01401': 17,
        '2100601': 28,
    }[x]

cyto_expansion = 5
n_diameter = select_cellpose_n_diameter(plateID)

flow_threshold = .4
cellprob_threshold=0
min_size=150
max_size = 500

resample = True

#call cellpose on each image to segment the nuclei
model_name = "Ctn"
model_path = os.path.join(data_path, "cellpose_training/train/models",model_name) #@param {type:"string"}

# DEFINE CELLPOSE MODEL
# declare model
model = models.CellposeModel(gpu=True, 
                             pretrained_model=model_path)
# define CHANNELS to run segementation on
# grayscale=0, R=1, G=2, B=3
# channels = [cytoplasm, nucleus]
# if NUCLEUS channel does not exist, set the second channel to 0
# will use channel R = 1 as nuclear channel only
n_channels = [0,0]

for subdir in subdirectories:
    results = [] #collect results for one field in the well
    field = re.findall("field_[1-9]",subdir)[0]
    field_num = re.findall("[0-9]", field)[0]
    reg_filename = os.path.join(registered_stacks_path,plateID+"_R_"+well+"_"+field_num+"_reg_stack.tif")
    mask_filename = os.path.join(output_path,plateID+"_"+well+"_"+field_num+"_nuc_masks.png")
    tracking_path = os.path.join(output_path,'tracking/')
    if not os.path.exists(tracking_path+well+"/"+field+"/nuc_masks/"):
            os.makedirs(tracking_path+well+"/"+field+"/nuc_masks/")
    if not os.path.exists(tracking_path+well+"/"+field+"/reg"):
            os.makedirs(tracking_path+well+"/"+field+"/reg")
    if not os.path.exists(tracking_path+well+"/"+field+"/results"):
            os.makedirs(tracking_path+well+"/"+field+"/results")
    if not os.path.exists(tracking_path+well+"/"+field+"/filtered_masks"):
            os.makedirs(tracking_path+well+"/"+field+"/filtered_masks")

    if not os.path.exists(mask_filename): #Only segment if no mask file
        print("Start segmenting "+reg_filename)
        img_rs_reg = io.imread(reg_filename)
        img_gs_reg = io.imread(reg_filename.replace('_R_', '_G_'))

        n_mask_images = []
        for i, image in enumerate(img_rs_reg):
            # create nuclear masks with cellpose 
            n_masks, flows, styles = model.eval(image,
                                                diameter=n_diameter,
                                                net_avg = True,
                                                flow_threshold=flow_threshold,
                                                cellprob_threshold=cellprob_threshold,
                                                channels=n_channels,
                                                min_size=min_size)
            #n_masks = morphology.remove_small_objects(n_masks, min_size)
            n_masks_filtered = np.copy(n_masks)
            component_sizes = np.bincount(n_masks.ravel())
            debris = np.logical_or(component_sizes < min_size, component_sizes > max_size)
            debris_mask = debris[n_masks]
            n_masks_filtered[debris_mask] = 0
            #mask_morphologies = measure.regionprops_table(n_masks_filtered, 
            #                                  properties=('label','area'))
            #grow masks and measure the reporter in the cytoplasm
            #add cytoplasm measurement to nuclear mask area
            #expand the masks to get cytoplasmic regions
            #nuclei_boundaries = segmentation.find_boundaries(n_masks, mode='thick')*n_masks
            #nuclei_expansions = segmentation.expand_labels(n_masks, cyto_expansion) - n_masks + nuclei_boundaries

            # measure cytoplasmic intensities in the green channel
            #nuclei_exp_g = measure.regionprops_table(nuclei_expansions, intensity_image=img_gs_reg[i],
            #                                         properties=('label','mean_intensity'))

            #cell_labels = mask_morphologies['label'][np.logical_and(mask_morphologies['area'] > min_size, mask_morphologies['area'] < max_size)]
            
            #replace any label that's too small with a 0 value
            #cell_labels = np.array([x if x in cell_labels
            #                           else 0 for x in range(0, n_masks.max()+1)])
 
            #n_masks_filtered = np.uint16(cell_labels[n_masks])

            #Save mask image
            n_mask_images.append(n_masks_filtered)
            
        tifffile.imwrite(mask_filename, np.array(n_mask_images), imagej=True)

        # Open the mask stack:
        im = Image.open(mask_filename)
 
        # create an index variable:
        i =0
        app = []
 
        # iterate over the mask stack and save each frame to disk:
        print("Save individual nuclear masks to disk for "+reg_filename)
        for fr in ImageSequence.Iterator(im):
            app.append(fr)
            fr.save(tracking_path+well+"/"+field+"/nuc_masks/"+"mask%03.d.tif"%i)
            i = i + 1
            
        # Open the registered intensity stack: use the phase images
        im = Image.open(reg_filename)
 
        # create an index variable:
        i = 0
        app = []
 
        # iterate over the registered phase stack and save each frame to disk:
        for fr in ImageSequence.Iterator(im):
            app.append(fr)
            fr.save(tracking_path+well+"/"+field+"/reg/"+"t%03.d.tif"%i)
            i = i + 1 

#### Track the nuclei  
Use the KIT-Loeffler tracking method to track the nuclei 
The tracking output includes masks with new label values and a res_track.txt file as described below  

##### Parameters  
delta_t = fill in missing masks for two sequential frames  
roi_size = consider cells across sequential frames that have moved by 2x the average of the segmentation masks bounding boxes as potential matches

#### Tracking method  
The Loeffler et al. tracking method is used to associated nuclei across the images, to identify progeny after mitosis and apoptosis. The default parameters of delta_t = 3 and roi_size = 2 are used. Objects that are not in the first or last two frames, are not parents and only last for 1 or 2 frames are considered debris and are removed from the data.  

In [24]:
for subdir in subdirectories: #track all fileds in the well
    field = re.findall("field_[1-9]",subdir)[0]
    #Condition on the tracking output file res_tracks.txt existing
    if not os.path.exists(tracking_path+well+"/"+str(field)+"/results/res_track.txt"): #Only track if no res_track.txt
        cmd = "python -m run_tracking --image_path "+ tracking_path+well+"/"+str(field)+"/reg/ --segmentation_path "+tracking_path+well+"/"+str(field)+"/nuc_masks/ --results_path "+tracking_path+well+"/"+str(field)+"/results --delta_t 3 --default_roi_size 2"
        returned_value = os.system(cmd)  # returns the exit code in unix
        print('returned value from command line tracking with KIT:', returned_value)


####################
time point: 0
####################
####################
time point: 1
####################
####################
time point: 2
####################
####################
time point: 3
####################
Add vertices to graph
Add sink and source vertex to graph
Set up constraints
Add Equations
Optimize
Optimal objective: 1500.25
####################
time point: 4
####################
####################
time point: 5
####################
####################
time point: 6
####################
Add vertices to graph
Add sink and source vertex to graph
Set up constraints
Add Equations
Optimize
Optimal objective: 3416.96
####################
time point: 7
####################
####################
time point: 8
####################
####################
time point: 9
####################
Add vertices to graph
Add sink and source vertex to graph
Set up constraints
Add Equations
Optimize
Optimal objective: 3722.84
####################
time point: 10
####################
##

#### Identify cells
Read in the tracking results  
res_track.txt - A text file representing an acyclic graph for the whole image sequence. Every line corresponds to a single track that is encoded by four numbers separated by a space:  
L B E P where  
L - a unique label of the track (label of markers, 16-bit positive value)  
B - a zero-based temporal index of the frame in which the track begins  
E - a zero-based temporal index of the frame in which the track ends  
P - label of the parent track (0 is used when no parent is defined)


Filter the track objects keeping the parents and those with a minimum track length and save the results to tracks.csv    

Create a new file tracks.csv with the following columns:  
label - a unique label of the track (label of markers, 16-bit positive value)  
begins - a zero-based temporal index of the frame in which the track begins  
ends - a zero-based temporal index of the frame in which the track ends  
parent - label of the parent track (0 is used when no parent is defined)  
length - The number of frames that the cell is identified in  
plateID - Character string of the plate's ID such as AU02001  
well - Character string of the well such as A1  
field - Integer of the image field within the well  

In [25]:
#set filter parameters
min_track_length = 2
#loop through the results from each segmented field
for subdir in subdirectories:
    field = re.findall("field_[1-9]",subdir)[0]
    res_filename = os.path.join(output_path,"tracking",well,field,"results","res_track.txt")
    res_flt_filename = res_filename.replace("res_track.txt","tracks.csv")
    tracks = pd.read_csv(res_filename, sep=" ", header=None)
    tracks.columns = ["label", "begins", "ends", "parent"]
    tracks['length'] = tracks.ends - tracks.begins + 1
    last_track = tracks.ends.max()
    #check if object is a parent
    tracks["is_parent"] = tracks['label'].isin(tracks['parent'])
    tracks['plateID'] = plateID
    tracks['well'] = well
    tracks['field'] = field.replace("field_","") #only filter if no tracks.csv file
    #If filtered results do not exist, read in the res_track.txt file for the current field
    if not os.path.exists(res_flt_filename):
        #Filter using the filter parameters
        #remove short tracks that are not parents and are not in the last frame
        tracks_flt = tracks.query('length >= @min_track_length or is_parent or ends > (@last_track-@min_track_length) or begins < 2' )
        ##remove any track that appears after the first frame and doesn't have a parent
        #tracks_flt = tracks_flt.query('not (begins > 1 & parent == 0)')
        #write out the res_flt_track.txt file
        tracks_flt.to_csv(res_flt_filename,index=False) 

#### Filter masks to only tracked cells  
Use the filtered tracks to remove masks for non-cell objects  
Save the filtered masks as individual image files in filtered_masks directory   

In [26]:
#loop through the fields in the well
for subdir in subdirectories:
    field = re.findall("field_[1-9]",subdir)[0]
    mask_track_path = os.path.join(output_path,"tracking",well,field,"results")
    tracked_mask_filenames = sorted(glob.glob(mask_track_path+"/mask*"))
    #condition on whether the filtered masks exist
    if not os.path.exists(tracked_mask_filenames[0].replace("results","filtered_masks")):
       #read in the tracks file for this field    
        res_flt_filename = os.path.join(output_path,"tracking",well,field,"results","tracks.csv")
        tracks = pd.read_csv(res_flt_filename) 
        #loop through the mask images in the field
        for fn in tracked_mask_filenames:
            #read in the mask image
            im = io.imread(fn)
           #replace any label that's not a cell with a 0 value
            cell_labels = np.array([x if x in tracks.label.to_numpy()
                                       else 0 for x in range(0, im.max()+1)])
            im_filtered = cell_labels[im]
            io.imsave(fn.replace("results","filtered_masks"), im_filtered.astype(np.int16), plugin='tifffile', check_contrast=False)

#### Get excel metadata file  
If this file does not exists, create a level 0 file that is data only  

In [27]:
#If the metadata exists, load it
metadata_filename = os.path.join(data_path,plateID,"metadata",plateID+".xlsx")

if os.path.exists(metadata_filename):
    md_all = pd.read_excel(metadata_filename, engine='openpyxl', dtype={'Drug1Concentration': str, 'Drug2Concentration': str})
    
    #remove unwanted columns read in from the excel files
    r = re.compile("Unnamed.*")
    columns_to_drop = list(filter(r.match, md_all.columns)) 
    metadata = md_all.drop(columns = columns_to_drop)
    
    #match metadata and data well labels format
    metadata['row'] = [re.sub(r'[0-9]*', '', Well) for Well in metadata['Well']]
    metadata['column'] = [re.sub(r'[A-Z]', '', Well) for Well in metadata['Well']]
    metadata['column'] = [re.sub(r'\A0', '', row) for row in metadata['column']]
    metadata['well'] = metadata['row'] + metadata['column']
    

#### Pull data from images  
Apply the filtered masks to the registered red nuclear channel and recored each cells nuclear morphology, intensity and texture  
Create cytoplasm masks by expanding the nuclear masks by a fixed amount or until they collide with another nuclear expansion  
Use the nuclear and cytoplasmic masks to measure intensities in the green fluroescent images  
Calculate intensity ratios between the cytomplasm and nuclei  
If the metadata is available, merge it with the cell level data  
Store the cell level data (and metadata) in a csv file where each row is a cell  
Data feature values can be decoded as follows:  
\<compartment>\_\<pipeline name>\_\<channel name>\_\<regionprops name>  
compartment - Nuclei, Cyto or Cell  
pipeline name - PC for python cellpose or other if added  
channel name - NR for nuclear reporter, CC for cell cycle reporter or others if added  
regionprops name - label passed through from the skimage measure.regionprops function https://scikit-image.org/docs/dev/api/skimage.measure.html#skimage.measure.regionprops  

#### Data extraction method  
The nuclear masks are expanded by 5 pixels to create masks to measure intensities in the cytoplasm. Both sets of masks are applied to the green channel cell cycle reporter images using the python skimage function regionprops_table to extract intensity and nuclar morphology values. Ratios between the cytoplasm and nuclear cell cycle reporter are calculated and the cells are gated on this ratio to be in G1 or S/G2 states. 

The cell level data is mean summarized to the population level for each image and the proportion of cells in G1 are calculated.

In [28]:
minutes_between_images = 30

def select_threshold(x):
    return {
        'AU00601': 0.97,
        'AU01501': 0.94,
        'AU01601': 0.94,
        'AU01701': 0.94,
        'AU01702': 0.94,
        'AU01801': 0.94,
        'AU01802': 0.94,
        'AU01901': 0.94,
        'AU01902': 0.94,
        'AU02001': 0.94,
        'AU02002': 0.94,
        'AU02101': 0.94,
        'AU02201': 0.94,
        'AU02301': 0.94,
        'AU02201': 0.94,
        'AU02401': 0.94,
        'AU02501': 0.94,
        'AU02601': 0.94,
        'AU02701': 0.94,
        'AU02801': 0.94,
        'AU02901': 0.94,
        'AU03001': 0.94,
        'AU03101': 0.94,
        'AU03201': 0.94,
        'AU03301': 0.94,
        'AU03401': 0.94,
        'AU03501': 0.94,
        'AU03601': 0.94,
        'AU03701': 0.94,
        'HC00701': 0.65,
        'HC00801': 0.74,
        'HC00901': 0.75,
        'HC01001': 0.76,
        'HC01301': 0.76,
        'HC01401': 0.76,
        '2100601': 0.85,
    }[x]

g1_threshold = select_threshold(plateID)

neighborhood_nuclei_distance = 5
neighborhood_radius_near = 20
neighborhood_radius_medium = 45
neighborhood_radius_far = 70

def calc_G1_prop(x):
    if('G1' in x.unique()):
        G1_prop = x.value_counts(normalize = True)['G1']
    else:
        G1_prop = 0
    return G1_prop

#loop through the fields in the well
for subdir in subdirectories:
    field = re.findall("field_[1-9]",subdir)[0]
    l0_filename = os.path.join(data_path+plateID,"Analysis",pipeline_name,plateID+"_"+well+"_"+field+"_level_0.csv")

    #condition on whether the l1 and l2 files exist
    if not os.path.exists(l0_filename.replace('level_0','level_1'))&os.path.exists(l0_filename.replace('level_0','level_2')):
        print("Pulling data from images "+l0_filename.replace('_level_0.csv',''))
        filtered_mask_path = os.path.join(output_path,"tracking",well,field,"filtered_masks")
        tracked_mask_filenames = sorted(glob.glob(filtered_mask_path+"/mask*"))
        img_gs_reg = io.imread(os.path.join(registered_stacks_path,plateID+"_G_"+well+"_"+field.replace("field_","")+"_reg_stack.tif"))
        # iterate over the mask files
        results = []
        for i, fn in enumerate(tracked_mask_filenames):
            #read in the mask image
            masks = io.imread(fn)
            #read in registered R images
            reg_fn = fn.replace("filtered_masks","reg")
            image = io.imread(reg_fn.replace("mask","t"))

            #measure reporter intensity and nuclear morphology, texture
            nuclei = measure.regionprops_table(masks, intensity_image=image,
                                               properties=('label',
                                                           'area','bbox_area','convex_area','centroid','eccentricity','equivalent_diameter','extent','feret_diameter_max','filled_area',
                                                            'major_axis_length','minor_axis_length','moments_hu','perimeter','perimeter_crofton','solidity',
                                                            'mean_intensity','max_intensity','min_intensity'))
            #expand the masks to get cytoplasmic regions
            nuclei_boundaries = segmentation.find_boundaries(masks, mode='thick')*masks
            nuclei_expansions = segmentation.expand_labels(masks, cyto_expansion) - masks + nuclei_boundaries

            # measure nuclear and cytoplasmic intensities and textures in the green channel
            nuclei_g = measure.regionprops_table(masks, intensity_image=img_gs_reg[i],
                                                 properties=('label','centroid',
                                                             'mean_intensity','max_intensity','min_intensity'))
            nuclei_exp_g = measure.regionprops_table(nuclei_expansions, intensity_image=img_gs_reg[i],
                                                     properties=('label','centroid',
                                                                 'mean_intensity','max_intensity','min_intensity'))

            # turn results into a dataframe
            nuclei_data = pd.DataFrame(nuclei)
            nuclei_data.rename(columns={col: 'Nuclei_'+pipeline_name+'_' +ch1_name+'_'+col  for col in nuclei_data.columns if col not in ['label']}, inplace=True)

            nuclei_g_data = pd.DataFrame(nuclei_g)
            nuclei_g_data.rename(columns={col: 'Nuclei_'+pipeline_name+'_' +ch2_name+'_'+col  for col in nuclei_g_data.columns if col not in ['label']}, inplace=True)

            nuclei_exp_g_data = pd.DataFrame(nuclei_exp_g)
            nuclei_exp_g_data.rename(columns={col: 'Cyto_'+pipeline_name+'_' +ch2_name+'_'+col  for col in nuclei_exp_g_data.columns if col not in ['label']}, inplace=True)

            # recover the well and field values and add them to the dataframe
            well = re.findall('/[A-Z][0-9]+/',reg_fn)[0]
            well = re.sub('/','', well)
            nuclei_data['well'] = well
            field = re.findall('field_[0-9]+',reg_fn)[0]
            field = int(re.sub('field_','', field))
            nuclei_data['field'] = field
            nuclei_data['slice'] = i
            nuclei_data['elapsed_minutes'] = i*minutes_between_images #assumes time slice numbering starts at 1
            elapsed_minutes = i*minutes_between_images #assumes time slice numbering starts at 1
            day = np.floor(elapsed_minutes/(24*60)).astype(int)
            hour = np.floor((elapsed_minutes-day*(24*60))/60).astype(int)
            minute = np.floor(elapsed_minutes-day*(24*60)-hour*60).astype(int)
            day = str(day).zfill(2)
            hour = str(hour).zfill(2)
            minute = str(minute).zfill(2)
            nuclei_data['time_slice'] = day+"d"+hour+"h"+minute+"m"

            #Calculate ratio of ch2 cyto to nuclei intensities
            nuclei_exp_g_data['Cell_'+pipeline_name+'_' +ch2_name+'_mean_intensity_ratio'] = nuclei_exp_g_data['Cyto_'+pipeline_name+'_' +ch2_name+'_mean_intensity']/nuclei_g_data['Nuclei_'+pipeline_name+'_' +ch2_name+'_mean_intensity']
            nuclei_exp_g_data['Cell_'+pipeline_name+'_' +ch2_name+'_max_intensity_ratio'] = nuclei_exp_g_data['Cyto_'+pipeline_name+'_' +ch2_name+'_max_intensity']/nuclei_g_data['Nuclei_'+pipeline_name+'_' +ch2_name+'_max_intensity']
            nuclei_exp_g_data['Cell_'+pipeline_name+'_' +ch2_name+'_min_intensity_ratio'] = nuclei_exp_g_data['Cyto_'+pipeline_name+'_' +ch2_name+'_min_intensity']/nuclei_g_data['Nuclei_'+pipeline_name+'_' +ch2_name+'_min_intensity']

            #label cell states based on the reporter ratio
            nuclei_exp_g_data['cell_cycle_state'] = 'G1'
            mask = nuclei_exp_g_data['Cell_'+pipeline_name+'_' +ch2_name+'_mean_intensity_ratio'] > g1_threshold
            nuclei_exp_g_data.loc[mask, 'cell_cycle_state'] = 'S/G2'
            nuclei_exp_g_data['cell_cycle_state_threshold'] = g1_threshold

            #calculate the neighborhood density
            nuclei_kd = spatial.KDTree(nuclei_data[['Nuclei_CtnK_NR_centroid-0','Nuclei_CtnK_NR_centroid-1']])
            nuclei_data['neighborhood_'+str(neighborhood_radius_near)] = nuclei_kd.query_ball_point(nuclei_data[['Nuclei_CtnK_NR_centroid-0','Nuclei_CtnK_NR_centroid-1']],
                                                                                                    r = neighborhood_radius_near, return_sorted = True, return_length=True)
            nuclei_data['neighborhood_'+str(neighborhood_radius_medium)] = nuclei_kd.query_ball_point(nuclei_data[['Nuclei_CtnK_NR_centroid-0','Nuclei_CtnK_NR_centroid-1']],
                                                                                                    r = neighborhood_radius_medium, return_sorted = True, return_length=True)
            nuclei_data['neighborhood_'+str(neighborhood_radius_far)] = nuclei_kd.query_ball_point(nuclei_data[['Nuclei_CtnK_NR_centroid-0','Nuclei_CtnK_NR_centroid-1']],
                                                                                                    r = neighborhood_radius_far, return_sorted = True, return_length=True)
                        
            #merge the dataframes from the different channels
            df = pd.merge(nuclei_data, nuclei_g_data, how="left", on=["label"])
            df_all = pd.merge(df, nuclei_exp_g_data, how="left", on=["label"])

            # append this image's dataframe to the results list
            results.append(df_all)

        #concatenate all of the results from all images in the field
        l0_image = pd.concat(results)

        #join with the tracking results to get lineage, parent, frame length values
        tracks_filename = os.path.join(output_path,"tracking",well,"field_"+str(field),"results/tracks.csv")
        tracks = pd.read_csv(tracks_filename)
        l0 = pd.merge(l0_image, tracks, how="left", on=["label", "well", "field"])
        
        #label cell states based on the reporter ratio
        l0['cell_cycle_state'] = 'G1'
        mask = l0['Cell_CtnK_CC_mean_intensity_ratio'] > g1_threshold
        l0.loc[mask, 'cell_cycle_state'] = 'S/G2'

        if os.path.exists(metadata_filename):
            #merge data and metadata on well values
            l1= pd.merge(l0, metadata, how="left", on=["well"]).round(decimals=2)
            l1['treatment'] =  l1['Drug1']+'_'+l1['Drug1Concentration']+'_'+l1['Drug2']+'_'+l1['Drug2Concentration']
            #filter out first 9 slices
            #l1 = l1[l1['slice'] > 9]
            print("Writing "+l0_filename.replace('level_0','level_1') + " to disk")
            l1.to_csv(l0_filename.replace('level_0','level_1'), index = False)
            #summarize to image level
            l2 = l1.groupby(['plateID','well','field', 'time_slice', 'elapsed_minutes','Drug1','Drug1Concentration', 'Drug2', 'Drug2Concentration', 'treatment', 'cell_cycle_state_threshold']).agg(
                n = ('plateID', 'size'),
                length=('length', 'mean'),
                Nuclei_CtnK_NR_area=('Nuclei_CtnK_NR_area', 'mean'),
                Nuclei_CtnK_CC_mean_intensity=('Nuclei_CtnK_CC_mean_intensity', 'mean'),
                Cyto_CtnK_CC_mean_intensity=('Cyto_CtnK_CC_mean_intensity', 'mean'),
                Cell_CtnK_CC_mean_intensity_ratio=('Cell_CtnK_CC_mean_intensity_ratio', 'mean'),
                Cell_CtnK_CC_max_intensity_ratio=('Cell_CtnK_CC_max_intensity_ratio', 'mean'),
                Cell_CtnK_CC_min_intensity_ratio=('Cell_CtnK_CC_min_intensity_ratio', 'mean'),
                G1_proportion =('cell_cycle_state', calc_G1_prop),
                neighborhood_20 = ('neighborhood_'+str(neighborhood_radius_near), 'mean'), #TODO figure out how to summarize with dynamic name
                neighborhood_45 = ('neighborhood_'+str(neighborhood_radius_medium), 'mean'),
                neighborhood_70 = ('neighborhood_'+str(neighborhood_radius_far), 'mean')
            ).reset_index().round(decimals=2)
            l2['cell_cycle_state_threshold'] = g1_threshold

            print("Writing "+l0_filename.replace('level_0','level_2') + " to disk")
            l2.to_csv(l0_filename.replace('level_0','level_2'), index = False)
        else:
            print("no metadata file for "+plateID+" so creating level 0 file")
            l0 = l0.round(decimals=2)
            l0.to_csv(l0_filename, index = False)

Pulling data from images /home/exacloud/gscratch/HeiserLab/images/HC01401/Analysis/CtnK/HC01401_A1_field_1
Writing /home/exacloud/gscratch/HeiserLab/images/HC01401/Analysis/CtnK/HC01401_A1_field_1_level_1.csv to disk
Writing /home/exacloud/gscratch/HeiserLab/images/HC01401/Analysis/CtnK/HC01401_A1_field_1_level_2.csv to disk
Pulling data from images /home/exacloud/gscratch/HeiserLab/images/HC01401/Analysis/CtnK/HC01401_A1_field_2
Writing /home/exacloud/gscratch/HeiserLab/images/HC01401/Analysis/CtnK/HC01401_A1_field_2_level_1.csv to disk
Writing /home/exacloud/gscratch/HeiserLab/images/HC01401/Analysis/CtnK/HC01401_A1_field_2_level_2.csv to disk
Pulling data from images /home/exacloud/gscratch/HeiserLab/images/HC01401/Analysis/CtnK/HC01401_A1_field_3
Writing /home/exacloud/gscratch/HeiserLab/images/HC01401/Analysis/CtnK/HC01401_A1_field_3_level_1.csv to disk
Writing /home/exacloud/gscratch/HeiserLab/images/HC01401/Analysis/CtnK/HC01401_A1_field_3_level_2.csv to disk
Pulling data from i